In [1]:
import sys
import os
import numpy as np
import pandas as pd
import warnings
warnings.filterwarnings('ignore')

pd.set_option('display.max_columns', None)

# 현재 노트북 파일의 경로를 기준으로 프로젝트 루트 경로를 계산
project_root = os.path.abspath(os.path.join(os.getcwd(), '..', '..', '..', '..'))

# sys.path에 프로젝트 루트 경로가 없으면 추가
if project_root not in sys.path:
  sys.path.append(project_root)


In [2]:
from src.data.db_handler import DBHandler

db_handler = DBHandler(db_name="data_lake")

df = db_handler.fetch_data(table_name="kr_stock_daily_itemchartprice")

column_index = df.columns

df.info()
df

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 848507 entries, 0 to 848506
Data columns (total 39 columns):
 #   Column                   Non-Null Count   Dtype              
---  ------                   --------------   -----              
 0   id                       848507 non-null  int64              
 1   ticker                   848507 non-null  object             
 2   prdy_vrss                848507 non-null  object             
 3   prdy_vrss_sign           848507 non-null  object             
 4   prdy_ctrt                848507 non-null  object             
 5   stck_prdy_clpr           848507 non-null  object             
 6   acml_vol                 848507 non-null  object             
 7   acml_tr_pbmn             848507 non-null  object             
 8   hts_kor_isnm             848507 non-null  object             
 9   stck_prpr                848507 non-null  object             
 10  stck_shrn_iscd           848507 non-null  object             
 11  prdy_vol     

,id,ticker,prdy_vrss,prdy_vrss_sign,prdy_ctrt,stck_prdy_clpr,acml_vol,acml_tr_pbmn,hts_kor_isnm,stck_prpr,stck_shrn_iscd,prdy_vol,stck_mxpr,stck_llam,stck_oprc,stck_hgpr,stck_lwpr,stck_prdy_oprc,stck_prdy_hgpr,stck_prdy_lwpr,askp,bidp,prdy_vrss_vol,vol_tnrt,stck_fcam,lstn_stcn,cpfn,hts_avls,per,eps,pbr,itewhol_loan_rmnd_ratem,stck_bsop_date,stck_clpr,flng_cls_code,prtt_rate,mod_yn,revl_issu_reas,updated_at
0,1,000080,-700,5,NaN,NaN,210028,7831937700,NaN,NaN,NaN,NaN,NaN,NaN,37700,38100,36900,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,None,20100108,36900,00,0.00,N,,2025-09-21 13:30:49.553216+00:00
1,2,000080,-1550,5,NaN,NaN,354686,13505883000,NaN,NaN,NaN,NaN,NaN,NaN,39650,39650,37500,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,None,20100107,37600,00,0.00,N,,2025-09-21 13:30:49.553216+00:00
2,3,000080,-1050,5,NaN,NaN,193081,7621324550,NaN,NaN,NaN,NaN,NaN,NaN,40200,40200,39150,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,None,20100106,39150,00,0.00,N,,2025-09-21 13:30:49.553216+00:00
3,4,000080,250,2,NaN,NaN,116732,4667762000,NaN,NaN,NaN,NaN,NaN,NaN,39900,40200,39650,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,None,20100105,40200,00,0.00,N,,2025-09-21 13:30:49.553216+00:00
4,5,000080,150,2,NaN,NaN,100021,3984604050,NaN,NaN,NaN,NaN,NaN,NaN,40100,40300,39600,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,None,20100104,39950,00,0.00,N,,2025-09-21 13:30:49.553216+00:00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
848502,848503,489790,-100,5,NaN,NaN,148461,8068900500,NaN,NaN,NaN,NaN,NaN,NaN,54700,55400,53800,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,None,20250822,54100,00,0.00,N,,2025-09-21 13:30:49.553216+00:00
848503,848504,489790,1100,2,NaN,NaN,237678,12875255150,NaN,NaN,NaN,NaN,NaN,NaN,53400,55000,53100,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,None,20250821,54200,00,0.00,N,,2025-09-21 13:30:49.553216+00:00
848504,848505,489790,-2200,5,NaN,NaN,369255,19603926650,NaN,NaN,NaN,NaN,NaN,NaN,53900,54100,52400,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,None,20250820,53100,00,0.00,N,,2025-09-21 13:30:49.553216+00:00
848505,848506,489790,-700,5,NaN,NaN,224713,12435825950,NaN,NaN,NaN,NaN,NaN,NaN,56300,56500,55000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,None,20250819,55300,00,0.00,N,,2025-09-21 13:30:49.553216+00:00


In [3]:
final_cols = column_index[1:]

In [4]:
df[final_cols] = df[final_cols].replace(r'^\s*$', np.nan, regex=True)
df[final_cols] = df[final_cols].replace(['0', 0], np.nan)

# 결측치 개수 계산
missing_value_counts = df[final_cols].isnull().sum()

# 전체 행의 개수 계산
total_rows = len(df[final_cols])

# 결측치 비율 계산
missing_value_percent = ((missing_value_counts / total_rows) * 100).round(2)


# --- 3. 결과 취합 및 출력 ---
missing_df = pd.DataFrame({
  'missing_count': missing_value_counts,
  'missing_percent': missing_value_percent
})

# 결측치가 많은 순서대로 정렬하여 출력
missing_df = missing_df[missing_df['missing_count'] > 0]
missing_df = missing_df.sort_values(by='missing_count', ascending=False)

missing_df

,missing_count,missing_percent
itewhol_loan_rmnd_ratem,848507,100.00
revl_issu_reas,848507,100.00
prdy_vrss,48120,5.67
acml_vol,2798,0.33
acml_tr_pbmn,2798,0.33
prdy_vrss_sign,2190,0.26


In [5]:
missing_cols = missing_df[missing_df['missing_percent'] >= 50].index.to_list()

final_cols = [col for col in final_cols if col not in missing_cols]

print(f"결측 컬럼 : {missing_cols}")

print(f"[원본 컬럼 : {len(column_index)-1}] - [결측 컬럼 : {len(missing_cols)}] = [후보 컬럼 : {len(final_cols)}]")

결측 컬럼 : ['itewhol_loan_rmnd_ratem', 'revl_issu_reas']
[원본 컬럼 : 38] - [결측 컬럼 : 2] = [후보 컬럼 : 36]


In [6]:
nunique_counts = df[final_cols].nunique()

# 전체 행의 개수 계산
total_rows = len(df[final_cols])

# 결측치 비율 계산
nunique_value_percent = ((nunique_counts / total_rows) * 100).round(2)

unique_cols = nunique_counts[nunique_counts == 1].index.tolist()

# --- 3. 결과 취합 및 출력 ---
nunique_df = pd.DataFrame({
  'nunique_count': nunique_counts,
  'nunique_percent': nunique_value_percent
})

# 결측치가 많은 순서대로 정렬하여 출력
nunique_df = nunique_df[nunique_df['nunique_count'] > 1]
nunique_df = nunique_df.sort_values(by='nunique_count', ascending=False)

nunique_df

,nunique_count,nunique_percent
acml_tr_pbmn,844553,99.53
acml_vol,529764,62.43
stck_hgpr,74368,8.76
stck_oprc,73412,8.65
stck_clpr,73299,8.64
stck_lwpr,73025,8.61
prdy_vrss,12333,1.45
stck_bsop_date,5365,0.63
ticker,200,0.02
prtt_rate,190,0.02


In [7]:
final_cols = [col for col in final_cols if col not in unique_cols]

print(f"유니크 컬럼 : {unique_cols}")

print(f"[원본 컬럼 : {len(column_index)-1}] - [결측 컬럼 : {len(missing_cols)}] - [유니크 컬럼 : {len(unique_cols)}] = [후보 컬럼 : {len(final_cols)}]")

유니크 컬럼 : ['prdy_ctrt', 'stck_prdy_clpr', 'hts_kor_isnm', 'stck_prpr', 'stck_shrn_iscd', 'prdy_vol', 'stck_mxpr', 'stck_llam', 'stck_prdy_oprc', 'stck_prdy_hgpr', 'stck_prdy_lwpr', 'askp', 'bidp', 'prdy_vrss_vol', 'vol_tnrt', 'stck_fcam', 'lstn_stcn', 'cpfn', 'hts_avls', 'per', 'eps', 'pbr', 'mod_yn', 'updated_at']
[원본 컬럼 : 38] - [결측 컬럼 : 2] - [유니크 컬럼 : 24] = [후보 컬럼 : 12]


In [8]:
df[final_cols]

,ticker,prdy_vrss,prdy_vrss_sign,acml_vol,acml_tr_pbmn,stck_oprc,stck_hgpr,stck_lwpr,stck_bsop_date,stck_clpr,flng_cls_code,prtt_rate
0,000080,-700,5,210028,7831937700,37700,38100,36900,20100108,36900,00,0.00
1,000080,-1550,5,354686,13505883000,39650,39650,37500,20100107,37600,00,0.00
2,000080,-1050,5,193081,7621324550,40200,40200,39150,20100106,39150,00,0.00
3,000080,250,2,116732,4667762000,39900,40200,39650,20100105,40200,00,0.00
4,000080,150,2,100021,3984604050,40100,40300,39600,20100104,39950,00,0.00
...,...,...,...,...,...,...,...,...,...,...,...,...
848502,489790,-100,5,148461,8068900500,54700,55400,53800,20250822,54100,00,0.00
848503,489790,1100,2,237678,12875255150,53400,55000,53100,20250821,54200,00,0.00
848504,489790,-2200,5,369255,19603926650,53900,54100,52400,20250820,53100,00,0.00
848505,489790,-700,5,224713,12435825950,56300,56500,55000,20250819,55300,00,0.00


In [9]:
print(final_cols)

['ticker', 'prdy_vrss', 'prdy_vrss_sign', 'acml_vol', 'acml_tr_pbmn', 'stck_oprc', 'stck_hgpr', 'stck_lwpr', 'stck_bsop_date', 'stck_clpr', 'flng_cls_code', 'prtt_rate']


| 컬럼명 | 한글컬럼명 | 중요도 | 이유 |
| :--- | :--- | :--- | :--- |
| `ticker` | 종목코드 | **필수** | 데이터를 종목별로 식별하는 Primary Key. |
| `prdy_vrss` | 전일 대비 등락 | **필수** | 주가 변동의 절대량을 나타내는 핵심 파생 피처. |
| `prdy_vrss_sign` | 전일 대비 부호 | **필수** | 주가 상승/하락 방향성을 나타내는 핵심 피처. |
| `acml_vol` | 누적 거래량 | **상** | 시장의 관심도와 유동성을 나타내는 핵심 지표. |
| `acml_tr_pbmn` | 누적 거래대금 | **상** | 거래량과 함께 시장의 자금 흐름을 판단하는 중요 지표. |
| `stck_oprc` | 시가 | **필수** | OHLC 데이터의 핵심 요소로, 당일 가격 흐름의 시작점. |
| `stck_hgpr` | 고가 | **필수** | OHLC 데이터의 핵심 요소로, 당일 변동성 및 저항선 분석에 사용. |
| `stck_lwpr` | 저가 | **필수** | OHLC 데이터의 핵심 요소로, 당일 변동성 및 지지선 분석에 사용. |
| `stck_bsop_date` | 영업일자 | **필수** | 시계열 분석의 기준이 되는 시간 축(time index). |
| `stck_clpr` | **종가** | **필수** | **예측 모델의 핵심 Target 또는 가장 중요한 피처.** |
| `flng_cls_code` | 락 구분 코드 | 중 | 권리락, 배당락 등 주가에 영향을 미치는 이벤트를 나타내는 피처. |
| `prtt_rate` | 등락률 | **필수** | 정규화된 주가 변동률로, 모델 학습에 직접 사용. |

In [10]:
final_cols = [
  'ticker',
  'prdy_vrss',
  'prdy_vrss_sign',
  'acml_vol',
  'acml_tr_pbmn',
  'stck_oprc',
  'stck_hgpr',
  'stck_lwpr',
  'stck_bsop_date',
  'stck_clpr',
  'flng_cls_code',
  'prtt_rate'
]

In [11]:
df[final_cols]

,ticker,prdy_vrss,prdy_vrss_sign,acml_vol,acml_tr_pbmn,stck_oprc,stck_hgpr,stck_lwpr,stck_bsop_date,stck_clpr,flng_cls_code,prtt_rate
0,000080,-700,5,210028,7831937700,37700,38100,36900,20100108,36900,00,0.00
1,000080,-1550,5,354686,13505883000,39650,39650,37500,20100107,37600,00,0.00
2,000080,-1050,5,193081,7621324550,40200,40200,39150,20100106,39150,00,0.00
3,000080,250,2,116732,4667762000,39900,40200,39650,20100105,40200,00,0.00
4,000080,150,2,100021,3984604050,40100,40300,39600,20100104,39950,00,0.00
...,...,...,...,...,...,...,...,...,...,...,...,...
848502,489790,-100,5,148461,8068900500,54700,55400,53800,20250822,54100,00,0.00
848503,489790,1100,2,237678,12875255150,53400,55000,53100,20250821,54200,00,0.00
848504,489790,-2200,5,369255,19603926650,53900,54100,52400,20250820,53100,00,0.00
848505,489790,-700,5,224713,12435825950,56300,56500,55000,20250819,55300,00,0.00


In [12]:
final_df = df[final_cols].copy()

# --- 1단계: 날짜 타입 자동 변환 ---
remaining_object_cols = final_df.select_dtypes(include='object').columns.tolist()
for col in remaining_object_cols:
  original_valid_count = final_df[col].count()
  if original_valid_count == 0: continue
  
  for format in ['%Y%m%d', '%Y-%m-%d', '%Y%m', '%Y-%m']:
    date_series = pd.to_datetime(final_df[col], format=format,errors='coerce')
    success_rate = date_series.count() / original_valid_count
    
    if success_rate > 0.95: # 95% 임계값
      final_df[col] = date_series
      break

# --- 2단계: 저비율 카테고리 자동 변환 ---
object_cols = final_df.select_dtypes(include='object').columns.tolist()
for col in object_cols:
  ratio = final_df[col].nunique() / len(final_df[col])
  if ratio < 0.05: # 5% 임계값
    final_df[col] = final_df[col].astype('category')


In [13]:
final_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 848507 entries, 0 to 848506
Data columns (total 12 columns):
 #   Column          Non-Null Count   Dtype         
---  ------          --------------   -----         
 0   ticker          848507 non-null  category      
 1   prdy_vrss       800387 non-null  category      
 2   prdy_vrss_sign  846317 non-null  category      
 3   acml_vol        845709 non-null  object        
 4   acml_tr_pbmn    845709 non-null  object        
 5   stck_oprc       848507 non-null  object        
 6   stck_hgpr       848507 non-null  object        
 7   stck_lwpr       848507 non-null  object        
 8   stck_bsop_date  848507 non-null  datetime64[ns]
 9   stck_clpr       848507 non-null  object        
 10  flng_cls_code   848507 non-null  category      
 11  prtt_rate       848507 non-null  category      
dtypes: category(5), datetime64[ns](1), object(6)
memory usage: 52.2+ MB


| 컬럼명 | 현재 타입 | 적정 타입 | 이유 |
| :--- | :--- | :--- | :--- |
| `ticker` | category | category | 최적 타입. 종목 식별자이므로 category 타입 유지. |
| `prdy_vrss` | category | int64 | 가격 변동폭은 정수형 데이터이므로 int64로 변환 필요. |
| `prdy_vrss_sign` | category | category | 최적 타입. 코드성 데이터이므로 category 타입 유지. |
| `acml_vol` | object | int64 | 누적 거래량은 큰 정수이므로 int64로 변환. |
| `acml_tr_pbmn` | object | int64 | 누적 거래대금은 매우 큰 정수이므로 int64로 변환. |
| `stck_oprc` | object | int64 | 시가 가격은 정수형 데이터. |
| `stck_hgpr` | object | int64 | 고가 가격은 정수형 데이터. |
| `stck_lwpr` | object | int64 | 저가 가격은 정수형 데이터. |
| `stck_bsop_date` | datetime64[ns] | datetime64[ns] | 최적 타입. 날짜 데이터이므로 타입 유지. |
| `stck_clpr` | object | int64 | 종가 가격은 정수형 데이터. |
| `flng_cls_code` | category | category | 최적 타입. 코드성 데이터이므로 category 타입 유지. |
| `prtt_rate` | category | float64 | 소수점을 포함하는 등락률 데이터이므로 float64로 변환 필요. |

In [14]:
cols_to_check = []

# 1. 모든 컬럼을 숫자 타입으로 먼저 변환
df[cols_to_check] = df[cols_to_check].apply(pd.to_numeric, errors='coerce')


# 2. 각 컬럼별로 정수 형태 데이터의 비율을 계산하는 함수를 적용
def get_integer_like_ratio(series: pd.Series) -> float:
  """
  Series의 유효한 값(NaN 제외) 중, 정수 형태인 값의 비율을 계산합니다.
  """
  # NaN 값을 제외한 데이터만 추출
  series_without_na = series.dropna()
  
  # 유효한 데이터가 없는 경우, 1.0 (100%) 반환
  if series_without_na.empty:
    return 1.0
    
  # (정수 형태인 값의 개수) / (전체 유효 데이터의 개수)
  integer_like_count = (series_without_na % 1 == 0).sum()
  total_valid_count = len(series_without_na)
  
  return integer_like_count / total_valid_count

# --- 결과 확인 ---
integer_like_ratio_result = df[cols_to_check].apply(get_integer_like_ratio)
print(integer_like_ratio_result.round(2))

Series([], dtype: float64)


In [15]:
# df.astype()에 사용하여 타입을 변경하기 위한 맵
# 이미 적정 타입인 컬럼은 제외됨
dtype_map = {
    'Int64': ['prdy_vrss', 'acml_vol', 'acml_tr_pbmn', 'stck_oprc', 'stck_hgpr', 'stck_lwpr', 'stck_clpr'],
    'float64': ['prtt_rate']
}

for dtype, cols in dtype_map.items():
  # DataFrame에 실제 존재하는 컬럼만 필터링
  valid_cols = [col for col in cols if col in final_df.columns]
  if not valid_cols:
    continue

  try:
    if dtype == 'Int64':
      for col in valid_cols:
        numeric_col = pd.to_numeric(final_df[col], errors='coerce')
        if not numeric_col.isnull().all():
          final_df[col] = numeric_col.astype('Int64')
    
    elif dtype == 'float64':
      final_df[valid_cols] = final_df[valid_cols].apply(pd.to_numeric, errors='coerce')

    elif dtype == 'datetime64':
      for col in valid_cols:
        final_df[col] = pd.to_datetime(final_df[col], errors='coerce')
    
    else: # 'category' 등 .astype()으로 처리 가능한 나머지 타입
      final_df[valid_cols] = final_df[valid_cols].astype(dtype)

  except (ValueError, TypeError) as e:
    print(f"⚠️ 경고: 컬럼 {valid_cols}을(를) '{dtype}'으로 변환 중 오류. 건너뜁니다. (에러: {e})")


In [16]:
final_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 848507 entries, 0 to 848506
Data columns (total 12 columns):
 #   Column          Non-Null Count   Dtype         
---  ------          --------------   -----         
 0   ticker          848507 non-null  category      
 1   prdy_vrss       800387 non-null  Int64         
 2   prdy_vrss_sign  846317 non-null  category      
 3   acml_vol        845709 non-null  Int64         
 4   acml_tr_pbmn    845709 non-null  Int64         
 5   stck_oprc       848507 non-null  Int64         
 6   stck_hgpr       848507 non-null  Int64         
 7   stck_lwpr       848507 non-null  Int64         
 8   stck_bsop_date  848507 non-null  datetime64[ns]
 9   stck_clpr       848507 non-null  Int64         
 10  flng_cls_code   848507 non-null  category      
 11  prtt_rate       848507 non-null  float64       
dtypes: Int64(7), category(3), datetime64[ns](1), float64(1)
memory usage: 67.2 MB
